In [1]:
%run preprocess_functions.ipynb

In [9]:
import os
import cv2
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch import nn
from torch.utils.data import Dataset
import torchvision.models as models
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split

In [ ]:
DIRECTORY_PATH = '/selected_images/'

# multi label for two eye input

In [2]:
class CustomDataset(Dataset):
    def __init__(self, dataset_X, dataset_y, image_directory_path, transfrom=None):
        self.X = dataset_X
        self.y = dataset_y
        self.image_directory_path = image_directory_path
        # self.grouped_by_id_data = self.dataset.groupby('patien_id') #groups in format of { patient_id: [indx1, indx2] }
        self.transform = transfrom or transforms.Compose([
            transforms.Lambda(preprocess),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) #values from image net, can be changed to dataset values
        ])
    def __len__(self):
        return len(self.X)
    def __getitem__(self, index):
        left_image = cv2.imread(os.path.join(self.image_directory_path, self.dataset.iloc[index]['image_id_left']))
        right_image = cv2.imread(os.path.join(self.image_directory_path, self.dataset.iloc[index]['image_id_right']))
        left_eye_diagnosis = self.y.iloc[index][['diabetic_retinopathy_left', 'amd_left', 'hypertensive_retinopathy_left', 
                                                'normal_eye_left', 'glaucoma_left', 'cataract_left']].to_numpy(dtype=np.float64)
        right_eye_diagnosis = self.y.iloc[index][['diabetic_retinopathy_right', 'amd_right', 'hypertensive_retinopathy_right', 
                                                 'normal_eye_right', 'glaucoma_right', 'cataract_right']].to_numpy(dtype=np.float64)
        age = self.datset.iloc[index]['patient_age']
        sex = self.dataset.iloc[index]['patient_sex']
        if self.transform:
           left_image = self.transfrom(left_image)
           right_image = self.transform(right_image)
        data = {"left_eye_image":left_image,
                "right_eye_image": right_image,
                "left_eye_diagnosis": torch.tensor(left_eye_diagnosis),
                "right_eye_diagnosis": torch.tensor(right_eye_diagnosis),
                "age and sex": torch.tensor(np.array([age, sex]))}
        return data

In [33]:
class NetworkVGG(nn.Module):
    def __init__(self):
        super(NetworkVGG, self).__init__()
        
        self.left_eye_input = models.vgg16(pretrained=True)
        self.right_eye_input = models.vgg16(pretrained=True)
        self.left_eye_input = nn.Sequential(*list(self.left_eye_input.children())[:-1])
        self.right_eye_input = nn.Sequential(*list(self.right_eye_input.children())[:-1])
        
        self.metadata_input = nn.Sequential(
            nn.Linear(2, 2),
            nn.ReLU()
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(4096 + 4096 + 2, 512),
            nn.ReLu(),
            nn.Linear(512, 6),
            nn.Sigmoid()
        )
        
    def forward(self, left_image, right_image, metadata):
        left_eye_vgg = self.left_eye_input(left_image)
        right_eye_vgg = self.right_eye_input(right_image)
        meta = self.metadata_input(metadata)
        concatenated = torch.cat((left_eye_vgg, right_eye_vgg, meta), dim=1)
        
        return self.classifier(concatenated)

In [ ]:
def train(dataloader, model, loss_function, optimizer):
    size = len(dataloader.dataset)
    running_loss = 0
    for batch, data in enumerate(dataloader):
        left_eye = data['left_eye_image']
        right_eye = data['right_eye']
        meta = data['age and sex']
        
        prediction = model(left_eye, right_eye, meta)
        
        loss = loss_function(prediction,)
        

In [23]:
!pip install torchsummary

  Obtaining dependency information for torchsummary from https://files.pythonhosted.org/packages/7d/18/1474d06f721b86e6a9b9d7392ad68bed711a02f3b61ac43f13c719db50a6/torchsummary-1.5.1-py3-none-any.whl.metadata



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [31]:
model = models.vgg16()
model.classifier = nn.Sequential(*list(model.children())[:-1])
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [30]:
model = models.vgg16()
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [3]:
Data = pd.read_csv("final_dataset.csv")
X = Data[['patient_age', 'patient_sex', 'image_id_left', 'image_id_right']]
y = Data[['diabetic_retinopathy_left', 'amd_left', 'hypertensive_retinopathy_left', 'normal_eye_left', 'glaucoma_left', 'cataract_left',
          'diabetic_retinopathy_right', 'amd_right', 'hypertensive_retinopathy_right', 'normal_eye_right', 'glaucoma_right', 'cataract_right']]
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


,patient_id,patient_age,patient_sex,data_source,image_id_left,diabetic_retinopathy_left,amd_left,hypertensive_retinopathy_left,normal_eye_left,glaucoma_left,cataract_left,image_id_right,diabetic_retinopathy_right,amd_right,hypertensive_retinopathy_right,normal_eye_right,glaucoma_right,cataract_right
0,11,21.0,2,BRSET,img00021.jpg,1,0,0,0,0,0,img00022.jpg,1,0,0,0,0,0
1,13,24.0,2,BRSET,img00025.jpg,1,0,0,0,0,0,img00026.jpg,1,0,0,0,0,0
2,25,28.0,2,BRSET,img00049.jpg,1,0,0,0,0,0,img00050.jpg,1,0,0,0,0,0
3,31,27.0,1,BRSET,img00060.jpg,1,0,0,0,0,0,img00061.jpg,1,0,0,0,0,0
4,38,37.0,1,BRSET,img00074.jpg,1,0,0,0,0,0,img00075.jpg,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2863,13211,63.0,1,ODIR,4686_right.jpg,1,0,0,0,0,0,4686_left.jpg,1,0,0,0,0,0
2864,13213,42.0,1,ODIR,4688_right.jpg,1,0,0,0,0,0,4688_left.jpg,1,0,0,0,0,0
2865,13214,54.0,1,ODIR,4689_right.jpg,0,0,0,1,0,0,4689_left.jpg,1,0,0,0,0,0
2866,13215,57.0,1,ODIR,4690_right.jpg,1,0,0,0,0,0,4690_left.jpg,1,0,0,0,0,0


In [26]:
torch.tensor(Data.iloc[1][['diabetic_retinopathy_left', 'amd_left', 'hypertensive_retinopathy_left', 'normal_eye_left', 'glaucoma_left', 'cataract_left']].to_numpy(dtype=np.float64))

tensor([1., 0., 0., 0., 0., 0.], dtype=torch.float64)

In [ ]:
# left_eye = dataset_nn[dataset_nn['exam_eye'] == 1]
# right_eye = dataset_nn[dataset_nn['exam_eye'] == 2]
# X_left = left_eye[['image_id', 'patient_id', 'patient_age', 'patient_sex']]
# y = dataset_nn[['diabetic_retinopathy', 'amd', 'hypertensive_retinopathy', 'normal_eye', 'glaucoma', 'cataract']]